<center><h1>Step 0 - Preprocessing</h1></center> 

In this section, we first read the data including the bug reports and source code files of all 51 projects and for ease of access, we save them as two pickle files in the ./Data directory. Therefore, this set of code will populate the ./Data directory with "allBugReports.pickle" which is a pandas dataframe that contains all the bug reports from all projects and "allSourceCodes.pickle" which is a pandas dataframe that contains all source files after preprocessing.

### Required Libraries

In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext('local[*]')

Picked up JAVA_TOOL_OPTIONS: -Xmx2g
Picked up JAVA_TOOL_OPTIONS: -Xmx2g
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 15:18:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from __future__ import division
import pandas as pd
# import ray.data as ray_pd
# import modin.pandas as ray_pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
import warnings
import javalang
import re
import glob
import math
import time
from scipy import spatial
import scipy.spatial.distance
import xml.etree.ElementTree as ET
import requests
import multiprocessing
from tqdm import tqdm_notebook
from time import gmtime, strftime
from random import randint
warnings.simplefilter(action='ignore', category=FutureWarning)
from pathlib import Path
import zlib
import pathlib
import time
from ast import literal_eval

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StringType, IntegerType, StructType,StructField,IntegerType,Row
from pyspark.sql.functions import udf, col

<center><h1>Splitting code and natural language</h1></center> 

In [5]:

def split_natural_lang(doc):
    """
    @Receives: a document in natural language (bugreport)
    @Process: splits it as described in BugLocator
    @Return: a list of lower cased words
    """
    try:
        wordList=[]
        word=''
        for char in doc:
            if char.isalnum() or char=='\'':
                word+=char
            else:
                if len(word)>0:
                    wordList.append(word)
                    word=''
        if len(word)>0:
            wordList.append(word)
        return wordList
    except:
        return []


def code_splitter(sourceCode):
    """
    @Receives: a code
    @Process: splits it same as described in BugLocator
    @Return: a list of lower cased words
    """
    contentBuf = []
    wordBuf = []
    for char in sourceCode:
        if ((char >= 'a' and char <= 'z') or (char >= 'A' and char <= 'Z')):
            wordBuf.append(char)
            continue
        length = len(wordBuf)
        if (length != 0):
            k = 0
            for i in range(length-1):
                j=i+1
                first_char = wordBuf[i]
                second_char = wordBuf[j]
                if ((first_char >= 'A' and first_char <= 'Z') and (second_char >= 'a' and second_char <= 'z')):
                    contentBuf.append(wordBuf[k:i])
                    contentBuf.append(' ')
                    k = i
                    continue
                if ((first_char >= 'a' and first_char <= 'z') and (second_char >= 'A' and second_char <= 'Z')):
                    contentBuf.append(wordBuf[k:j])
                    contentBuf.append(' ')
                    k = j
                    continue
            if (k < length):
                contentBuf.append(wordBuf[k:])
                contentBuf.append(" ")
            wordBuf=[]
    words=''
    for each in contentBuf:
        if isinstance(each,str):
            words+=each
        else: 
            for term in each: 
                words+=term
    words= words.split()
    contentBuf = []
    for i in range(len(words)):
        if (words[i].strip()!="" and len(words[i]) >= 2):
            contentBuf.append(words[i])
    return contentBuf



def general_preprocessor(doc,mode):
    """
    @Receives: a document (code or bug report denoted by mode)
    @Process: processes the docucument by stemming and removing stop-words and converting to lower cases
    @Return: a list of lower cased words
    """
    JavaKeywords=["abstract", "continue", "for", 
                "new", "switch", "assert", "default", "goto", "package", 
                "synchronized", "boolean", "do", "if", "private", "this", 
                "break", "double", "implements", "protected", "throw", "byte", 
                "else", "import", "public", "throws", "case", "enum", 
                "instanceof", "return", "transient", "catch", "extends", "int", 
                "short", "try", "char", "final", "interface", "static", "void", 
                "class", "finally", "long", "strictfp", "volatile", "const", 
                "float", "native", "super", "while", "org", "eclipse", "swt", 
                "string", "main", "args", "null", "this", "extends", "true", 
                "false"]
    stop_words=["a", "a's", "able", "about", "above",
                "according", "accordingly", "across", "actually", "after",
                "afterwards", "again", "against", "ain't", "all", "allow",
                "allows", "almost", "alone", "along", "already", "also",
                "although", "always", "am", "among", "amongst", "an", "and",
                "another", "any", "anybody", "anyhow", "anyone", "anything",
                "anyway", "anyways", "anywhere", "apart", "appear",
                "appreciate", "appropriate", "are", "aren't", "around", "as",
                "aside", "ask", "asking", "associated", "at", "available",
                "away", "awfully", "b", "be", "became", "because", "become",
                "becomes", "becoming", "been", "before", "beforehand",
                "behind", "being", "believe", "below", "beside", "besides",
                "best", "better", "between", "beyond", "both", "brief", "but",
                "by", "c", "c'mon", "c's", "came", "can", "can't", "cannot",
                "cant", "cause", "causes", "certain", "certainly", "changes",
                "clearly", "co", "com", "come", "comes", "concerning",
                "consequently", "consider", "considering", "contain",
                "containing", "contains", "corresponding", "could", "couldn't",
                "course", "currently", "d", "definitely", "described",
                "despite", "did", "didn't", "different", "do", "does",
                "doesn't", "doing", "don't", "done", "down", "downwards",
                "during", "e", "each", "edu", "eg", "eight", "either", "else",
                "elsewhere", "enough", "entirely", "especially", "et", "etc",
                "even", "ever", "every", "everybody", "everyone", "everything",
                "everywhere", "ex", "exactly", "example", "except", "f", "far",
                "few", "fifth", "first", "five", "followed", "following",
                "follows", "for", "former", "formerly", "forth", "four",
                "from", "further", "furthermore", "g", "get", "gets",
                "getting", "given", "gives", "go", "goes", "going", "gone",
                "got", "gotten", "greetings", "h", "had", "hadn't", "happens",
                "hardly", "has", "hasn't", "have", "haven't", "having", "he",
                "he's", "hello", "help", "hence", "her", "here", "here's",
                "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
                "hi", "him", "himself", "his", "hither", "hopefully", "how",
                "howbeit", "however", "i", "i'd", "i'll", "i'm", "i've", "ie",
                "if", "ignored", "immediate", "in", "inasmuch", "inc",
                "indeed", "indicate", "indicated", "indicates", "inner",
                "insofar", "instead", "into", "inward", "is", "isn't", "it",
                "it'd", "it'll", "it's", "its", "itself", "j", "just", "k",
                "keep", "keeps", "kept", "know", "knows", "known", "l", "last",
                "lately", "later", "latter", "latterly", "least", "less",
                "lest", "let", "let's", "like", "liked", "likely", "little",
                "look", "looking", "looks", "ltd", "m", "mainly", "many", 
                "may", "maybe", "me", "mean", "meanwhile", "merely", "might",
                "more", "moreover", "most", "mostly", "much", "must", "my",
                "myself", "n", "name", "namely", "nd", "near", "nearly",
                "necessary", "need", "needs", "neither", "never",
                "nevertheless", "new", "next", "nine", "no", "nobody", "non",
                "none", "noone", "nor", "normally", "not", "nothing", "novel",
                "now", "nowhere", "o", "obviously", "of", "off", "often", "oh",
                "ok", "okay", "old", "on", "once", "one", "ones", "only",
                "onto", "or", "other", "others", "otherwise", "ought", "our",
                "ours", "ourselves", "out", "outside", "over", "overall",
                "own", "p", "particular", "particularly", "per", "perhaps",
                "placed", "please", "plus", "possible", "presumably",
                "probably", "provides", "q", "que", "quite", "qv", "r",
                "rather", "rd", "re", "really", "reasonably", "regarding",
                "regardless", "regards", "relatively", "respectively", "right",
                "s", "said", "same", "saw", "say", "saying", "says", "second",
                "secondly", "see", "seeing", "seem", "seemed", "seeming",
                "seems", "seen", "self", "selves", "sensible", "sent",
                "serious", "seriously", "seven", "several", "shall", "she",
                "should", "shouldn't", "since", "six", "so", "some",
                "somebody", "somehow", "someone", "something", "sometime",
                "sometimes", "somewhat", "somewhere", "soon", "sorry",
                "specified", "specify", "specifying", "still", "sub", "such",
                "sup", "sure", "t", "t's", "take", "taken", "tell", "tends",
                "th", "than", "thank", "thanks", "thanx", "that", "that's",
                "thats", "the", "their", "theirs", "them", "themselves",
                "then", "thence", "there", "there's", "thereafter", "thereby",
                "therefore", "therein", "theres", "thereupon", "these", "they",
                "they'd", "they'll", "they're", "they've", "think", "third",
                "this", "thorough", "thoroughly", "those", "though", "three",
                "through", "throughout", "thru", "thus", "to", "together",
                "too", "took", "toward", "towards", "tried", "tries", "truly",
                "try", "trying", "twice", "two", "u", "un", "under",
                "unfortunately", "unless", "unlikely", "until", "unto", "up",
                "upon", "us", "use", "used", "useful", "uses", "using",
                "usually", "uucp", "v", "value", "various", "very", "via",
                "viz", "vs", "w", "want", "wants", "was", "wasn't", "way",
                "we", "we'd", "we'll", "we're", "we've", "welcome", "well",
                "went", "were", "weren't", "what", "what's", "whatever",
                "when", "whence", "whenever", "where", "where's", "whereafter",
                "whereas", "whereby", "wherein", "whereupon", "wherever",
                "whether", "which", "while", "whither", "who", "who's",
                "whoever", "whole", "whom", "whose", "why", "will", "willing",
                "wish", "with", "within", "without", "won't", "wonder",
                "would", "would", "wouldn't", "x", "y", "yes", "yet", "you",
                "you'd", "you'll", "you're", "you've", "your", "yours",
                "yourself", "yourselves", "z", "zero","quot"]
    
    porter = PorterStemmer()
    Java_keyWords=[porter.stem(each.strip().lower()) for each in JavaKeywords]
    natural_stop_words=[porter.stem(each.strip().lower()) for each in stop_words]
    
    processed_doc=[]
    if mode=="code":
        splitted_doc=[porter.stem(term.lower()) for term in code_splitter(doc)]
        processed_doc=[term for term in splitted_doc if not(term in Java_keyWords or
                                                            term in natural_stop_words or len(term)<2)]
    elif mode=="text":
        splitted_doc=[porter.stem(term.lower()) for term in split_natural_lang(doc)]
        processed_doc=[term for term in splitted_doc if not(term in natural_stop_words or len(term)<2)]
    return processed_doc

<center><h1>Loading source codes into pandas Dataframe</h1></center> 

In [21]:
def classNames_methodNames(node):
    result=''
    if isinstance(node,javalang.tree.MethodDeclaration) or isinstance(node,javalang.tree.ClassDeclaration):
        return node.name.lower()+' '
    if not (isinstance(node,javalang.tree.PackageDeclaration) or
        isinstance(node,javalang.tree.FormalParameter) or
       isinstance(node,javalang.tree.Import)):
        if node:
            if isinstance(node, javalang.ast.Node):
                for childNode in node.children:
                    result+=classNames_methodNames(childNode)
    return result
    
def traverse_node(node,i=0):
    i+=1
    result=''
    if not(isinstance(node,javalang.tree.PackageDeclaration)
            or isinstance(node,javalang.tree.FormalParameter)            
            or isinstance(node,javalang.tree.Import)
            or isinstance(node,javalang.tree.CompilationUnit)):
        if node:
            if (isinstance(node,int) or isinstance(node,str) or isinstance(node,float)) and i==2:
                result+=node+' '
            if isinstance(node, javalang.ast.Node):
                for childNode in node.children:
                    result+=traverse_node(childNode,i)
    return result

def code_parser(code):
    try:
        tree = javalang.parse.parse(code)
        return ''.join([traverse_node(node) for path, node in tree]) + ' ' + ''.join([classNames_methodNames(node)
                                                                                      for path, node in tree])
    except Exception as e: 
        # print(e)
        return ''

    
def async_sourcecode_procesing(source_code):
#     print(source_code)
#     try:
    sc_code = zlib.decompress(bytes.fromhex(source_code)).decode()
#     except:
#         print(source_code)
# #     print(sc_code)
    processed_code=general_preprocessor(code_parser(sc_code),'code')
#     print(processed_code)
    return Row('file_content_processed', 'size')(zlib.compress("-|-".join(processed_code).encode("utf-8")).hex(), len(processed_code))

<center><h1>Loading bug reports pandas Dataframe</h1></center> 

In [7]:
def loadBugs2df(report):
#     print(report)
    return general_preprocessor(report,"text")


In [8]:
spark = SparkSession \
    .builder \
    .appName("BL") \
    .config("spark.some.config.option", "") \
    .getOrCreate()

In [27]:
all_bug_df_spark_df = spark.read.option("escape","\"").option('header', True).option('inferSchema', True).option('delimiter', '\t').csv("../globug_bl_dataset/combined_dataset/allBugReports.csv")

In [28]:
all_bug_df_spark_df.show()

+---+---+------+--------------------+--------------------+-------------------+----------------+--------------+-------+----------------+--------------------+--------------------+------------+--------------------+----------+------------+
|_c0| id|bug_id|             summary|         description|        report_time|report_timestamp|        status| commit|commit_timestamp|               files|         Unnamed: 10|project_name|              report|Unnamed: 0|Unnamed: 0.1|
+---+---+------+--------------------+--------------------+-------------------+----------------+--------------+-------+----------------+--------------------+--------------------+------------+--------------------+----------+------------+
|  0|  1|423257|Bug 423257 LTW - ...|                null|2013-12-04 19:43:22|      1386200000|resolved fixed|dd88d21|      1386350000|org.aspectj.ajdt....|52:tests/src/org/...|     AspectJ|Bug 423257 LTW - ...|      null|        null|
|  1|  2|420210|Bug 420210 Suppor...|It would be good ..

22/03/02 12:32:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
 Schema: _c0, id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
Expected: _c0 but found: 
CSV file: file:///scratch/varumuga/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allBugReports.csv


In [24]:
all_bug_df_spark_df.select('project_name').distinct().collect()

[Row(project_name='AspectJ'),
 Row(project_name='Eclipse_Platform_UI'),
 Row(project_name='JDT'),
 Row(project_name='SWT'),
 Row(project_name='Birt'),
 Row(project_name='Tomcat')]

In [29]:
all_bug_df_spark_df.where(all_bug_df_spark_df.project_name.isNull()).show()


22/03/02 12:32:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
 Schema: _c0, id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
Expected: _c0 but found: 
CSV file: file:///scratch/varumuga/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allBugReports.csv


+---+---+------+-------+-----------+-----------+----------------+------+------+----------------+-----+-----------+------------+------+----------+------------+
|_c0| id|bug_id|summary|description|report_time|report_timestamp|status|commit|commit_timestamp|files|Unnamed: 10|project_name|report|Unnamed: 0|Unnamed: 0.1|
+---+---+------+-------+-----------+-----------+----------------+------+------+----------------+-----+-----------+------------+------+----------+------------+
+---+---+------+-------+-----------+-----------+----------------+------+------+----------------+-----+-----------+------------+------+----------+------------+



In [30]:
udf_br_processor = udf(lambda x: general_preprocessor(x, "text"), ArrayType(StringType())) # if the function returns an int
df = all_bug_df_spark_df.withColumn("report_processed", udf_br_processor(col("report"))) #"_3" being the column name of the column you want to consider

In [31]:
 %timeit 
(df.show())

22/03/02 12:32:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
 Schema: _c0, id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
Expected: _c0 but found: 
CSV file: file:///scratch/varumuga/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allBugReports.csv


+---+---+------+--------------------+--------------------+-------------------+----------------+--------------+-------+----------------+--------------------+--------------------+------------+--------------------+----------+------------+--------------------+
|_c0| id|bug_id|             summary|         description|        report_time|report_timestamp|        status| commit|commit_timestamp|               files|         Unnamed: 10|project_name|              report|Unnamed: 0|Unnamed: 0.1|    report_processed|
+---+---+------+--------------------+--------------------+-------------------+----------------+--------------+-------+----------------+--------------------+--------------------+------------+--------------------+----------+------------+--------------------+
|  0|  1|423257|Bug 423257 LTW - ...|                null|2013-12-04 19:43:22|      1386200000|resolved fixed|dd88d21|      1386350000|org.aspectj.ajdt....|52:tests/src/org/...|     AspectJ|Bug 423257 LTW - ...|      null|       

In [32]:
df.toPandas().to_csv("../globug_bl_dataset/combined_dataset/allBugReportsProcessed.csv", sep="\t")

22/03/02 12:33:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
 Schema: _c0, id, bug_id, summary, description, report_time, report_timestamp, status, commit, commit_timestamp, files, Unnamed: 10, project_name, report, Unnamed: 0, Unnamed: 0.1
Expected: _c0 but found: 
CSV file: file:///scratch/varumuga/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allBugReports.csv


In [34]:
all_br_pandas_df = pd.read_csv("../globug_bl_dataset/combined_dataset/allBugReportsProcessed.csv", sep="\t")

In [ ]:
all_br_pandas_df.report_processed = all_br_pandas_df.report_processed.apply(literal_eval)

In [37]:
all_br_pandas_df.head()

,Unnamed: 0,_c0,id,bug_id,summary,description,report_time,report_timestamp,status,commit,commit_timestamp,files,Unnamed: 10,project_name,report,Unnamed: 0.1,Unnamed: 0.1.1,report_processed
0,0,0,1,423257,Bug 423257 LTW - java.lang.VerifyError: Bad re...,NaN,2013-12-04 19:43:22,1386200000,resolved fixed,dd88d21,1386350000,org.aspectj.ajdt.core/src/org/aspectj/ajdt/int...,52:tests/src/org/aspectj/systemtest/incrementa...,AspectJ,Bug 423257 LTW - java.lang.VerifyError: Bad re...,NaN,NaN,"[bug, 423257, ltw, java, lang, verifyerror, ba..."
1,1,1,2,420210,Bug 420210 Support additional message insert k...,It would be good to be able to insert the encl...,2013-10-23 16:00:16,1382560000,resolved fixed,9319e34,1382560000,org.aspectj.matcher/src/org/aspectj/weaver/Che...,63:org.aspectj.matcher/src/org/aspectj/weaver/...,AspectJ,Bug 420210 Support additional message insert k...,NaN,NaN,"[bug, 420210, support, addit, messag, insert, ..."
2,2,2,3,419279,Bug 419279 ajc option to change -Xlint level p...,The -Xlintfile option is not a great fit for c...,2013-10-11 15:20:08,1381520000,resolved fixed,b2cd5fa,1382560000,org.aspectj.ajdt.core/src/org/aspectj/ajdt/int...,6:org.aspectj.ajdt.core/src/org/aspectj/ajdt/i...,AspectJ,Bug 419279 ajc option to change -Xlint level p...,NaN,NaN,"[bug, 419279, ajc, option, xlint, level, messa..."
3,3,3,4,415266,Bug 415266 LTW not working when JMX is enabled,When I enable JMX remote management on a JVM a...,2013-08-16 17:51:29,1376690000,resolved fixed,9e992d6,1380650000,loadtime/src/org/aspectj/weaver/loadtime/Aj.java,5:loadtime/src/org/aspectj/weaver/loadtime/Aj....,AspectJ,Bug 415266 LTW not working when JMX is enabled...,NaN,NaN,"[bug, 415266, ltw, work, jmx, enabledwhen, ena..."
4,4,4,5,418129,Bug 418129 Can't introduce annotation onto int...,NaN,2013-09-26 14:31:09,1380220000,resolved fixed,2393bef,1380650000,tests/bugs174/pr418129/Target.java tests/bugs1...,21:weaver/src/org/aspectj/weaver/bcel/BcelClas...,AspectJ,Bug 418129 Can't introduce annotation onto int...,NaN,NaN,"[bug, 418129, introduc, annot, introduc, metho..."


In [38]:
all_br_pandas_df.files = all_br_pandas_df.files.apply(lambda x: x.split())

In [39]:
all_br_pandas_df.project_name.unique()

array(['AspectJ', 'Eclipse_Platform_UI', 'JDT', 'SWT', 'Birt', 'Tomcat'],
      dtype=object)

In [40]:
all_br_pandas_df[all_br_pandas_df.project_name == "AspectJ"].files.iloc[0]

['org.aspectj.ajdt.core/src/org/aspectj/ajdt/internal/core/builder/AjState.java',
 'tests/bugs175/pr423257/AspectX.java',
 'tests/bugs175/pr423257/Test.java',
 'tests/src/org/aspectj/systemtest/AllTests17.java',
 'tests/src/org/aspectj/systemtest/ajc175/Ajc175Tests.java',
 'tests/src/org/aspectj/systemtest/ajc175/AllTestsAspectJ175.java',
 'tests/src/org/aspectj/systemtest/incremental/tools/MultiProjectIncrementalTests.java',
 'weaver/src/org/aspectj/weaver/bcel/asm/StackMapAdder.java']

In [41]:
all_br_pandas_df.to_pickle("../globug_bl_dataset/combined_dataset/allBugReportsProcessed.pickle")

In [9]:
all_sc_df_spark_df = spark.read.option('header', True).option('inferSchema', True).option('delimiter', ',').csv("/home/varumuga/scratch/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allSourceCodes.csv")

In [12]:
all_sc_df_spark_df.select('project_name').distinct().collect()

[Row(project_name='AspectJ'),
 Row(project_name='Eclipse_Platform_UI'),
 Row(project_name='JDT'),
 Row(project_name='SWT'),
 Row(project_name='Birt'),
 Row(project_name='Tomcat')]

In [10]:
all_sc_df_spark_df.show()

+---+---+--------------------+------------+
|_c0|cid|        file_content|project_name|
+---+---+--------------------+------------+
|  0|  0|789cd5595b6fdb381...|     AspectJ|
|  1|  1|789c4bce492c2e567...|     AspectJ|
|  2|  2|789cd3d74fce492c2...|     AspectJ|
|  3|  3|789c4d8d310ec2301...|     AspectJ|
|  4|  4|789c2b284dcac94c5...|     AspectJ|
|  5|  5|789c8d93dd52c2301...|     AspectJ|
|  6|  6|789c258c3b0e80201...|     AspectJ|
|  7|  7|789ce53dfd571b479...|     AspectJ|
|  8|  8|789c7552b16ec2301...|     AspectJ|
|  9|  9|789c2b284dcac94c5...|     AspectJ|
| 10| 10|789c2b484cce4e4c4...|     AspectJ|
| 11| 11|789c4b2c2e484d2e5...|     AspectJ|
| 12| 12|789c558e416ac3301...|     AspectJ|
| 13| 13|789ca594cd6edb301...|     AspectJ|
| 14| 14|789c6d90b10e83201...|     AspectJ|
| 15| 15|789c2d8c310e40401...|     AspectJ|
| 16| 16|789c25cabb0d80300...|     AspectJ|
| 17| 17|789ced5a7b6f1a491...|     AspectJ|
| 18| 18|789c2b484cce4e4c4...|     AspectJ|
| 19| 19|789c7d8ebb0a02311...|  

22/03/02 15:21:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , cid, file_content, project_name
 Schema: _c0, cid, file_content, project_name
Expected: _c0 but found: 
CSV file: file:///home/varumuga/scratch/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allSourceCodes.csv


In [17]:
schema = StructType([
    StructField("file_content_processed", StringType(), False),
    StructField("size", IntegerType(), False)])

In [18]:
udf_br_processor = udf(async_sourcecode_procesing, schema) # if the function returns an int

In [35]:
# udf_br_processor = udf(lambda x: async_sourcecode_procesing(x), StringType()) # if the function returns an int
sc_processed_df = all_sc_df_spark_df.select("*").limit(10).withColumn("output", udf_br_processor(col("file_content"))) #"_3" being the column name of the column you want to consider

In [36]:
sc_processed_df = sc_processed_df.select("cid", "file_content", "output.file_content_processed", "output.size").limit(10)

In [37]:
sc_processed_df.write.mode("overwrite").parquet("/home/varumuga/scratch/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allSourceCodesProcessed")














































































[Stage 9:>  (0 + 16) / 47][Stage 10:>  (0 + 0) / 47][Stage 11:>  (0 + 0) / 47]


















































































ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/varumuga/replication/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/varumuga/replication/lib/python3.7/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.7.9/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 






Unterminated block comment at "
", line 2: /**





















































































Unterminated character/string literal at """, line 7: @Pointcut("call\n\n\n\n\n\n\n\n\n\n\n%dwdwudwdwbuill817pe;][{
Unterminated block comment at "
", line 7: /* foo














































































































Could not process token at "\", line 9: \



















































In [30]:
sc_processed_df.select().write.parquet("/home/varumuga/scratch/Thesis/Replications/Globug/globug_bl_dataset/combined_dataset/allSourceCodesProcessed")

[Stage 9:>                                                        (0 + 16) / 47]









































22/03/02 15:39:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers




22/03/02 15:39:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers



22/03/02 15:39:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers


22/03/02 15:39:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers

22/03/02 15:39:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
22/03/02 15:39:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes

KeyboardInterrupt: 




22/03/02 15:39:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 47.50% for 16 writers


















































































































[Stage 9:>                                                        (0 + 16) / 47]
































In [ ]:
all_bug_df_spark_processed_df = spark.read.option('header', True).option('inferSchema', True).option('delimiter', ',').csv("/home/varumuga/scratch/thesis/replication/bench_bl_dataset/data/allSourceCodesProcessed")

In [ ]:
all_bug_df_spark_processed_df.count()

In [ ]:
all_bug_df_spark_processed_df.show()

In [ ]:
def calculate_len(sc_compressed):
    sc_code = zlib.decompress(bytes.fromhex(sc_compressed)).decode()
    return len(sc_code.split("-|-"))

In [ ]:
udf_sc_size = udf(lambda x: calculate_len(x), IntegerType()) # if the function returns an int
df = all_bug_df_spark_processed_df.withColumn("size", udf_sc_size(col("file_content_processed"))) #"_3" being the column name of the column you want to consider

In [ ]:
df.show()

In [ ]:
df.write.option("header",True).option('delimiter', ',').csv("/home/varumuga/scratch/thesis/replication/bench_bl_dataset/data/allSourceCodesProcessedLength")

In [ ]:
df.filter(col("project_name") == "AspectJ").show()


<center><h1>Main Preprocessing class</h1></center> 

In [ ]:
class PreprocessingUnit:

    # all_projects_source_codes=pd.DataFrame([])
    # all_projects_bugreports=pd.DataFrame([])
    
    

    def __init__(self,project_name):
#         project_df = ray_pd.DataFrame([])
        self.dataPath=project_name
        # self.dataFolder=os.path.join(os.getcwd(),'Data')
        # if not os.path.exists(self.dataFolder):
        #     os.makedirs(self.dataFolder)
            
    def execute(self):

        self.loadEverything()

    def loadEverything(self):
#         vectorize=False
#         if PreprocessingUnit.all_projects_bugreports.empty:
#             bugReportFile=os.path.join(self.dataFolder,'allBugReports.csv')
#             if not os.path.isfile(bugReportFile):
#                 PreprocessingUnit.all_projects_bugreports=load_all_BRs(dataPath=self.dataPath)
#                 vectorize=True
#                 if vectorize:
#                     PreprocessingUnit.all_projects_bugreports.to_csv(bugReportFile, index=False)
#             else: 
#                 PreprocessingUnit.all_projects_bugreports=pd.read_pickle(bugReportFile)
# #         if vectorize:
# #             with open()
#         print("*** All bug reports are are preprocessed and stored as: {} ***".format('/'.join(bugReportFile.split('/')[-2:])))
        start = time.time()
        df1 = ray_pd.read_csv("{}.csv".format(self.dataPath), delimiter='\t')
        df2 = ray_pd.read_csv("{}_features.csv".format(self.dataPath))
        df3 = ray_pd.read_csv("{}_features_file_content.csv".format(self.dataPath))
        df4 = ray_pd.merge(df2, df3, left_on='cid', right_on='cid', how='inner')
        df5 = ray_pd.merge(df1, df4, left_on='id', right_on='report_id', how='inner')
        df5['report'] = df5['summary'] + df5['description']
        df5['project_name'] = self.dataPath.split("/")[-1]
        df5 = df5.head()
#         modin_df = from_pandas(df5)
        
#         df5 = df5.head()
#         ray_pd = ray_pd.from_pandas(df5)
        reports = df5.report.values.tolist()
        source_codes = df5.file_content.values.tolist()
        processed_reports = []
        processed_source_codes = []
        for i, r in enumerate(reports):
            if len(processed_reports) > 1000:
                num_ready = i-1000
                ray.wait(processed_reports, num_returns=num_ready)

            processed_reports.append(loadBugs2df.remote(r))
#             try:
#                 processed_reports.append(loadBugs2df(r))
#             except:
#                 processed_reports.append(None)
        for i, sc in enumerate(source_codes):
            if len(processed_source_codes) > 1000:
                    num_ready = i-1000
                    ray.wait(processed_source_codes, num_returns=num_ready)
            processed_source_codes.append(async_sourcecode_procesing.remote(sc))
# #             try:
# #                 processed_source_codes.append(async_sourcecode_procesing(sc))
# #             except:
# #                 processed_reports.append(None)
        processed_reports_ = []
        for pr in processed_reports:
            try:
                processed_reports_.append(ray.get(pr))
            except:
                processed_reports_.append(ray.get(None))
        processed_reports = processed_reports_
        processed_src_ = []
        for pr_src in processed_source_codes:
            try:
                processed_src_.append(ray.get(pr_src))
            except:
                processed_src_.append(ray.get(None))
        processed_reports = processed_reports_
        processed_source_codes = processed_src_
        df5["globug_processed_report"] = processed_reports #df5.report.apply(lambda x: loadBugs2df.remote(x))
        df5["globug_processed_file_content"] = processed_source_codes #df5.file_content.apply(lambda x: async_sourcecode_procesing.remote(x))

        processed_folder = os.path.join("/".join(self.dataPath.split("/")[:-2]), "processed_bl_reports")
        pathlib.Path(processed_folder).mkdir(parents=True, exist_ok=True)

        project = self.dataPath.split("/")[-1]
        project_processed_folder = os.path.join(processed_folder, project)
        pathlib.Path(project_processed_folder).mkdir(parents=True, exist_ok=True)
        project_processed_file = os.path.join(project_processed_folder, f"{project}_processed.csv")

        df5.to_csv(project_processed_file, index=False)
        end = time.time()
        print(f"Processing project: {project} completed")
            
        

In [ ]:
@ray.remote
def process_df(path):
    pu = PreprocessingUnit(path)
    pu.execute()
    return pu

In [ ]:
projects = ["Birt", "Tomcat", "JDT", "SWT", "Eclipse_Platform_UI"]
# projects = ["AspectJ"]

In [ ]:
processing = []
for p in projects:
    data_path = (f"/home/varumuga/scratch/thesis/replication/bench_bl_dataset/Dataset/{p}")
    processing.append(process_df.remote(data_path))

processing = ray.get(processing) 

### MAIN

In [ ]:
# df = pd.read_csv("/home/varumuga/scratch/thesis/replication/bench_bl_dataset/processed_bl_reports/AspectJ_processed.csv")

In [ ]:
# type(preprocessor.all_projects_bugreports)

In [ ]:
# preprocessor.dataFolder
# bugReportFile=os.path.join(preprocessor.dataFolder,'allBugReports.csv')

In [ ]:
# bugReportFile

In [ ]:
# PreprocessingUnit.all_projects_bugreports.to_csv(bugReportFile, index=False)

In [ ]:
# a = pd.read_pickle(bugReportFile)

In [ ]:
# a.head(10)

In [ ]:
# PreprocessingUnit.all_projects_bugreports.head()